In [ ]:
!pip install pysptk
!pip install -U path.py


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 461.9/461.9 kB 20.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pysptk: filename=pysptk-1.0.1-cp311-cp311-linux_x86_64.whl size=1293740 sha256=41666f712ac63038e244a2f2ee1527b4aa3e4ebd21643dd579981b07d8867c9e
  Stored in directory: /root/.cache/pip/wheels/f7/8e/e6/be2295ab3cba2d52e826922cd3ea8bda4a18210c03584dc759
Successfully built pysptk


In [ ]:
import numpy as np
import pandas as pd
import os
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm  # For progress bar in loops

from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (Input, Conv2D, MaxPooling2D, Flatten, Dense,
                                     Dropout, BatchNormalization, GlobalAveragePooling2D)
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical

# Google Colab specific imports
from google.colab import drive
drive.mount('/content/drive')
!pip install -U path.py
import os
from path import Path
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#df = pd.read_csv("IEMOCAP_Final.csv")
#csv_file_path = Path("/content/drive/MyDrive/Emotion Detection/Image/features_labeled.csv")

# Path to the CSV file
csv_file_path = Path("/content/drive/MyDrive/Emotion Detection/Image/IEMOCAP_Final.csv")
labels_df = pd.read_csv(csv_file_path)

# Create a dictionary mapping speaker_id to emotion
#emotion_dict = pd.Series(labels_df.emotion.values, index=labels_df.speaker_id).to_dict()
labels_df['dimension'][0]

"[{'arousal': 3.0, 'valence': 3.0, 'dominance': 4.0}]"

In [ ]:
df = labels_df[['Speaker_id','Transcript','dimension','category']].copy()
df['category'] = df['category'].astype(str).str.lower()
df = df.drop_duplicates(subset='Transcript')


In [ ]:
from collections import Counter
import ast


def get_majority_label(label_str):
    labels = ast.literal_eval(label_str)  # convert string to list
    count = Counter(labels)
    most_common = count.most_common(1)[0]
    if most_common[1] >= 2:
        return most_common[0]
    else:
        return None


# Create a new column for final emotion if majority exists
df['Emotion'] = df['category'].apply(get_majority_label)

# Drop rows where no majority
df_filtered = df.dropna(subset=['Emotion'])

df_filtered

,Speaker_id,Transcript,dimension,category,Emotion
2,Ses04F_script03_1_M001,I wonder.,"[{'arousal': 3.0, 'valence': 3.0, 'dominance':...","['fear', 'fear', 'excited']",fear
4,Ses04F_script03_1_M002,It must be them then.,"[{'arousal': 3.0, 'valence': 2.5, 'dominance':...","['fear', 'fear', 'excited']",fear
11,Ses04F_script03_1_F005,Ahh. It sent shivers up my spine.,"[{'arousal': 4.0, 'valence': 4.0, 'dominance':...","['frustration', 'excited', 'excited']",excited
12,Ses04F_script03_1_M006,What shall we do if they suddenly walk in on us?,"[{'arousal': 3.5, 'valence': 2.0, 'dominance':...","['frustration', 'excited', 'excited']",excited
13,Ses04F_script03_1_F006,Behave exquisitely.,"[{'arousal': 3.5, 'valence': 4.0, 'dominance':...","['excited', 'happiness', 'excited']",excited
...,...,...,...,...,...
10030,Ses05F_script01_3_M037,It felt wrong to be alive-to open that bank-bo...,"[{'arousal': 4.5, 'valence': 1.5, 'dominance':...","['frustration', 'sadness', 'sadness']",sadness
10031,Ses05F_script01_3_M038,"And if you're not, then it's all just loot and...","[{'arousal': 4.0, 'valence': 1.0, 'dominance':...","['frustration', 'sadness', 'frustration']",frustration
10033,Ses05F_script01_3_M039,"I I want you now, Annie.","[{'arousal': 4.0, 'valence': 3.0, 'dominance':...","['excited', 'sadness', 'excited']",excited
10034,Ses05F_script01_3_F025,Because you can't feel like that anymore Chris...,"[{'arousal': 3.5, 'valence': 2.5, 'dominance':...","['sadness', 'sadness', 'other']",sadness


In [ ]:
emotion_mapping = {
    'neutral': 'neutral',
    'frustration': 'angry',
    'anger': 'angry',
    'surprise': None,
    'disgust': None,
    'other': None,
    'sadness': 'sad',
    'fear': None,
    'happiness': 'happy',
    'excited': 'happy'
}

# Map the Emotion column
df_filtered['Mapped_Emotion'] = df_filtered['Emotion'].map(emotion_mapping)

# Drop rows where mapping returns None
df_final = df_filtered.dropna(subset=['Mapped_Emotion'])

df_final


<ipython-input-6-6852a1d54b7d>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['Mapped_Emotion'] = df_filtered['Emotion'].map(emotion_mapping)


,Speaker_id,Transcript,dimension,category,Emotion,Mapped_Emotion
11,Ses04F_script03_1_F005,Ahh. It sent shivers up my spine.,"[{'arousal': 4.0, 'valence': 4.0, 'dominance':...","['frustration', 'excited', 'excited']",excited,happy
12,Ses04F_script03_1_M006,What shall we do if they suddenly walk in on us?,"[{'arousal': 3.5, 'valence': 2.0, 'dominance':...","['frustration', 'excited', 'excited']",excited,happy
13,Ses04F_script03_1_F006,Behave exquisitely.,"[{'arousal': 3.5, 'valence': 4.0, 'dominance':...","['excited', 'happiness', 'excited']",excited,happy
14,Ses04F_script03_1_M007,With the most perfect poise.,"[{'arousal': 3.5, 'valence': 3.0, 'dominance':...","['excited', 'neutral', 'excited']",excited,happy
15,Ses04F_script03_1_F007,Certainly I shall probably do a court curtsy.,"[{'arousal': 3.5, 'valence': 3.5, 'dominance':...","['excited', 'happiness', 'excited']",excited,happy
...,...,...,...,...,...,...
10030,Ses05F_script01_3_M037,It felt wrong to be alive-to open that bank-bo...,"[{'arousal': 4.5, 'valence': 1.5, 'dominance':...","['frustration', 'sadness', 'sadness']",sadness,sad
10031,Ses05F_script01_3_M038,"And if you're not, then it's all just loot and...","[{'arousal': 4.0, 'valence': 1.0, 'dominance':...","['frustration', 'sadness', 'frustration']",frustration,angry
10033,Ses05F_script01_3_M039,"I I want you now, Annie.","[{'arousal': 4.0, 'valence': 3.0, 'dominance':...","['excited', 'sadness', 'excited']",excited,happy
10034,Ses05F_script01_3_F025,Because you can't feel like that anymore Chris...,"[{'arousal': 3.5, 'valence': 2.5, 'dominance':...","['sadness', 'sadness', 'other']",sadness,sad


In [ ]:
import pandas as pd
import ast
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error

# Step 1: parse 'dimension' column (handling both list and dict formats)
def smart_parse(x):
    try:
        if pd.isna(x):
            return None
        parsed = ast.literal_eval(x)
        if isinstance(parsed, list) and len(parsed) > 0 and isinstance(parsed[0], dict):
            return parsed[0]
        elif isinstance(parsed, dict):
            return parsed
    except Exception:
        return None


# Apply the smart parser
df_final['parsed_dim'] = df_final['dimension'].apply(smart_parse)
print("Valid rows:", df_final['parsed_dim'].notnull().sum())
print("Invalid rows:", df_final['parsed_dim'].isnull().sum())


# Filter only valid
df_clean = df_final.copy()

# Flatten VAD scores
vad_df = pd.json_normalize(df_clean['parsed_dim'])
df_clean = df_clean.reset_index(drop=True)
vad_df = vad_df.reset_index(drop=True)
# Combine with transcript
df_model = pd.concat([df_clean[['Transcript']], vad_df], axis=1)

# Final NaN check
print(df_model.isnull().sum())




Valid rows: 6336
Invalid rows: 0
Transcript    0
arousal       0
valence       0
dominance     0
dtype: int64


<ipython-input-7-a55cce7e8acf>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['parsed_dim'] = df_final['dimension'].apply(smart_parse)


In [ ]:
df_model

,Transcript,arousal,valence,dominance
0,Ahh. It sent shivers up my spine.,4.0,4.0,4.0
1,What shall we do if they suddenly walk in on us?,3.5,2.0,3.5
2,Behave exquisitely.,3.5,4.0,3.5
3,With the most perfect poise.,3.5,3.0,3.5
4,Certainly I shall probably do a court curtsy.,3.5,3.5,3.5
...,...,...,...,...
6331,It felt wrong to be alive-to open that bank-bo...,4.5,1.5,4.5
6332,"And if you're not, then it's all just loot and...",4.0,1.0,4.5
6333,"I I want you now, Annie.",4.0,3.0,3.5
6334,Because you can't feel like that anymore Chris...,3.5,2.5,4.5


# Experiment 1: 1st try with RF at Stage 1

In [ ]:
#Split data
X = df_model['Transcript']
y = df_model[['valence', 'arousal', 'dominance']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build pipeline (TF-IDF + MultiOutput Random Forest)
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=5000)),
    ('regressor', MultiOutputRegressor(RandomForestRegressor(n_estimators=100, random_state=42)))
])

#  Train
pipeline.fit(X_train, y_train)

# Predict
y_pred = pipeline.predict(X_test)

# Step 10: Evaluate
mse = mean_squared_error(y_test, y_pred, multioutput='raw_values')
print(f"Valence MSE: {mse[0]:.4f}, Arousal MSE: {mse[1]:.4f}, Dominance MSE: {mse[2]:.4f}")


Valence MSE: 0.5801, Arousal MSE: 0.4318, Dominance MSE: 0.5351


In [ ]:
from sklearn.metrics import r2_score

r2 = r2_score(y_test, y_pred, multioutput='raw_values')
print(f"Valence R²: {r2[0]:.4f}, Arousal R²: {r2[1]:.4f}, Dominance R²: {r2[2]:.4f}")


Valence R²: 0.3732, Arousal R²: 0.1800, Dominance R²: 0.1750


In [ ]:
#df_model[df_model['Transcript'].str.contains('Do you still feel like that?')]
#pd.json_normalize(df_final[df_final['Transcript'].str.contains('What would I do with a fortune?')])
#df_clean[df_clean['Transcript'].str.contains('What would I do with a fortune?')]

,Transcript,arousal,valence,dominance
8659,Do you still feel like that?,NaN,NaN,NaN


EXPERIMENT *1*: Stage 1 TEXT TO VAD USING REBERTA

In [ ]:
import torch
import numpy as np

BATCH_SIZE = 16
LEARNING_RATE = 2e-5
NUM_EPOCHS = 10
MAX_SEQ_LENGTH = 128
MODEL_NAME = "roberta-base"
RANDOM_SEED = 42

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

torch.manual_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)


Device: cuda


In [ ]:
from torch.utils.data import Dataset

class TextVADDataset(Dataset):
    def __init__(self, texts, vad_values, tokenizer, max_len=MAX_SEQ_LENGTH):
        self.texts = texts
        self.vad_values = vad_values
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        vad = self.vad_values[idx]

        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'vad_values': torch.tensor(vad, dtype=torch.float)
        }


In [ ]:
import torch.nn as nn
from transformers import AutoModel

class TextVADModel(nn.Module):
    def __init__(self, model_name=MODEL_NAME, dropout=0.1):
        super(TextVADModel, self).__init__()
        self.transformer = AutoModel.from_pretrained(model_name)
        hidden_size = self.transformer.config.hidden_size

        self.shared_layer = nn.Sequential(
            nn.Linear(hidden_size, 512),
            nn.LayerNorm(512),
            nn.Dropout(dropout),
            nn.GELU(),
        )

        self.valence_branch = nn.Sequential(
            nn.Linear(512, 128),
            nn.LayerNorm(128),
            nn.Dropout(dropout),
            nn.GELU(),
            nn.Linear(128, 1)
        )
        self.arousal_branch = nn.Sequential(
            nn.Linear(512, 128),
            nn.LayerNorm(128),
            nn.Dropout(dropout),
            nn.GELU(),
            nn.Linear(128, 1)
        )
        self.dominance_branch = nn.Sequential(
            nn.Linear(512, 128),
            nn.LayerNorm(128),
            nn.Dropout(dropout),
            nn.GELU(),
            nn.Linear(128, 1)
        )

    def forward(self, input_ids, attention_mask):
        outputs = self.transformer(input_ids=input_ids, attention_mask=attention_mask)
        pooled = outputs.last_hidden_state[:, 0, :]
        shared = self.shared_layer(pooled)
        valence = self.valence_branch(shared)
        arousal = self.arousal_branch(shared)
        dominance = self.dominance_branch(shared)
        return torch.cat([valence, arousal, dominance], dim=1)


In [ ]:
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer
from torch.utils.data import DataLoader

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Use df_model directly
texts = df_model['Transcript'].values
vad = df_model[['valence', 'arousal', 'dominance']].values

X_train, X_test, y_train, y_test = train_test_split(texts, vad, test_size=0.2, random_state=RANDOM_SEED)

train_dataset = TextVADDataset(X_train, y_train, tokenizer)
test_dataset = TextVADDataset(X_test, y_test, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup

model = TextVADModel().to(device)
criterion = nn.MSELoss()
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0.1 * len(train_loader) * NUM_EPOCHS,
    num_training_steps=len(train_loader) * NUM_EPOCHS
)
NUM_EPOCHS = 20
for epoch in range(NUM_EPOCHS):
    model.train()
    total_loss = 0
    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        targets = batch['vad_values'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{NUM_EPOCHS} - Loss: {total_loss/len(train_loader):.4f}")


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/20 - Loss: 2.7637
Epoch 2/20 - Loss: 0.6303
Epoch 3/20 - Loss: 0.5169
Epoch 4/20 - Loss: 0.4387
Epoch 5/20 - Loss: 0.3846
Epoch 6/20 - Loss: 0.3312
Epoch 7/20 - Loss: 0.2980
Epoch 8/20 - Loss: 0.2776
Epoch 9/20 - Loss: 0.2542
Epoch 10/20 - Loss: 0.2373
Epoch 11/20 - Loss: 0.2355
Epoch 12/20 - Loss: 0.2357
Epoch 13/20 - Loss: 0.2342
Epoch 14/20 - Loss: 0.2365
Epoch 15/20 - Loss: 0.2352
Epoch 16/20 - Loss: 0.2368
Epoch 17/20 - Loss: 0.2359
Epoch 18/20 - Loss: 0.2342
Epoch 19/20 - Loss: 0.2354
Epoch 20/20 - Loss: 0.2361


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

model.eval()
all_preds, all_targets = [], []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        targets = batch['vad_values'].to(device)

        outputs = model(input_ids, attention_mask)
        all_preds.append(outputs.cpu().numpy())
        all_targets.append(targets.cpu().numpy())

# Stack all predictions and targets
preds = np.vstack(all_preds)
targets = np.vstack(all_targets)

# Compute metrics per VAD dimension
mse = mean_squared_error(targets, preds, multioutput='raw_values')
rmse = np.sqrt(mse)
mae = mean_absolute_error(targets, preds, multioutput='raw_values')
r2 = r2_score(targets, preds, multioutput='raw_values')

# Print nicely
vad_labels = ['Valence', 'Arousal', 'Dominance']
for i in range(3):
    print(f"{vad_labels[i]} - MSE: {mse[i]:.4f}, RMSE: {rmse[i]:.4f}, MAE: {mae[i]:.4f}, R²: {r2[i]:.4f}")


Valence - MSE: 0.4329, RMSE: 0.6580, MAE: 0.4992, R²: 0.5322
Arousal - MSE: 0.4219, RMSE: 0.6495, MAE: 0.5112, R²: 0.1989
Dominance - MSE: 0.5022, RMSE: 0.7087, MAE: 0.5687, R²: 0.2256


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import f1_score

# Extract data
texts = df_clean['Transcript'].values
emotion_labels = df_clean['Mapped_Emotion'].values

\
X_train_texts, X_test_texts, y_train_labels, y_test_labels = train_test_split(
    texts, emotion_labels, test_size=0.2, random_state=42
)

# Stage 2 inputs
X_stage2 = preds  # predicted VADs from Stage 1
y_stage2 = y_test_labels  # true emotion labels

# Now split this into train/test for Stage 2 classifier


X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(
    X_stage2, y_stage2, test_size=0.2, random_state=42
)


clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train_2, y_train_2)

y_pred_2 = clf.predict(X_test_2)

print("Stage 2 Classification Report (Predicted VAD → Emotion):")
print(classification_report(y_test_2, y_pred_2))
print(f"Accuracy: {accuracy_score(y_test_2, y_pred_2) * 100:.2f}%")


weighted_f1 = f1_score(y_test_2, y_pred_2, average='weighted')
print(f"Weighted F1 Score: {weighted_f1:.4f}")
f1_macro = f1_score(y_test_2, y_pred_2, average='macro')
f1_micro = f1_score(y_test_2, y_pred_2, average='micro')


Stage 2 Classification Report (Predicted VAD → Emotion):
              precision    recall  f1-score   support

       angry       0.73      0.70      0.72       105
       happy       0.66      0.82      0.73        60
     neutral       0.40      0.40      0.40        57
         sad       0.45      0.31      0.37        32

    accuracy                           0.61       254
   macro avg       0.56      0.56      0.56       254
weighted avg       0.61      0.61      0.61       254

Accuracy: 61.42%
Weighted F1 Score: 0.6070


In [ ]:
y_stage2

array(['angry', 'angry', 'angry', ..., 'sad', 'angry', 'angry'],
      dtype=object)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, accuracy_score, classification_report

# Store models to test
models = {
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "SVM (Linear)": SVC(kernel='linear', probability=True),
    "KNN": KNeighborsClassifier(n_neighbors=5),
    "MLP (Neural Net)": MLPClassifier(hidden_layer_sizes=(128, 64), max_iter=1000),
    "Naive Bayes": GaussianNB()
}

# Train and evaluate each model
for name, model in models.items():
    model.fit(X_train_2, y_train_2)
    y_pred = model.predict(X_test_2)

    weighted_f1 = f1_score(y_test_2, y_pred, average='weighted')
    accuracy = accuracy_score(y_test_2, y_pred)

    print(f"\n**************{name}*******************")
    print(f"Accuracy: {accuracy:.2%}")
    print(f"Weighted F1 Score: {weighted_f1:.4f}")
    print(classification_report(y_test_2, y_pred))



**************Random Forest*******************
Accuracy: 61.42%
Weighted F1 Score: 0.6070
              precision    recall  f1-score   support

       angry       0.73      0.70      0.72       105
       happy       0.66      0.82      0.73        60
     neutral       0.40      0.40      0.40        57
         sad       0.45      0.31      0.37        32

    accuracy                           0.61       254
   macro avg       0.56      0.56      0.56       254
weighted avg       0.61      0.61      0.61       254


**************Logistic Regression*******************
Accuracy: 63.78%
Weighted F1 Score: 0.6159
              precision    recall  f1-score   support

       angry       0.70      0.78      0.74       105
       happy       0.67      0.87      0.75        60
     neutral       0.44      0.33      0.38        57
         sad       0.56      0.28      0.38        32

    accuracy                           0.64       254
   macro avg       0.59      0.57      0.56       2

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



**************MLP (Neural Net)*******************
Accuracy: 64.57%
Weighted F1 Score: 0.6214
              precision    recall  f1-score   support

       angry       0.73      0.79      0.76       105
       happy       0.66      0.90      0.76        60
     neutral       0.42      0.33      0.37        57
         sad       0.62      0.25      0.36        32

    accuracy                           0.65       254
   macro avg       0.61      0.57      0.56       254
weighted avg       0.63      0.65      0.62       254


**************Naive Bayes*******************
Accuracy: 64.96%
Weighted F1 Score: 0.6466
              precision    recall  f1-score   support

       angry       0.80      0.72      0.76       105
       happy       0.68      0.82      0.74        60
     neutral       0.45      0.53      0.48        57
         sad       0.50      0.31      0.38        32

    accuracy                           0.65       254
   macro avg       0.61      0.59      0.59       254
we

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Encode string labels to integers
label_encoder = LabelEncoder()
y_train_enc = label_encoder.fit_transform(y_train_2)
y_test_enc = label_encoder.transform(y_test_2)
from xgboost import XGBClassifier

xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)
xgb_clf.fit(X_train_2, y_train_enc)

y_pred_enc = xgb_clf.predict(X_test_2)

# Decode predictions back to original labels
y_pred = label_encoder.inverse_transform(y_pred_enc)
from sklearn.metrics import classification_report, f1_score, accuracy_score

print(" XGBoost Classifier (Encoded) Performance:")
print(classification_report(y_test_2, y_pred, zero_division=0))
print(f"Accuracy: {accuracy_score(y_test_2, y_pred) * 100:.2f}%")

weighted_f1 = f1_score(y_test_2, y_pred, average='weighted')
print(f"Weighted F1 Score: {weighted_f1:.4f}")


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [08:09:58] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


🔍 XGBoost Classifier (Encoded) Performance:
              precision    recall  f1-score   support

       angry       0.76      0.72      0.74       105
       happy       0.65      0.78      0.71        60
     neutral       0.42      0.39      0.40        57
         sad       0.34      0.31      0.33        32

    accuracy                           0.61       254
   macro avg       0.54      0.55      0.55       254
weighted avg       0.60      0.61      0.61       254

Accuracy: 61.02%
Weighted F1 Score: 0.6058


In [ ]:
from sklearn.model_selection import GridSearchCV

# Define model hyperparameter grids
param_grids = {
    "Random Forest": {
        'n_estimators': [100, 200],
        'max_depth': [None, 10, 20]
    },
    "Logistic Regression": {
        'C': [0.1, 1, 10],
        'solver': ['liblinear', 'lbfgs']
    },
    "SVM (Linear)": {
        'C': [0.1, 1, 10]
    },
    "KNN": {
        'n_neighbors': [3, 5, 7]
    },
    "MLP (Neural Net)": {
        'hidden_layer_sizes': [(64,), (128, 64)],
        'alpha': [0.0001, 0.001]
    },
    "Naive Bayes": {}  # No hyperparameters to tune for GaussianNB
}

models = {
    "Random Forest": RandomForestClassifier(random_state=42),
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "SVM (Linear)": SVC(kernel='linear', probability=True),
    "KNN": KNeighborsClassifier(),
    "MLP (Neural Net)": MLPClassifier(max_iter=1000),
    "Naive Bayes": GaussianNB()
}

# Train and evaluate with tuning
for name, model in models.items():
    print(f"\n************** {name} *******************")
    params = param_grids.get(name, {})

    if params:
        clf = GridSearchCV(model, params, cv=3, scoring='f1_weighted', n_jobs=-1)
        clf.fit(X_train_2, y_train_2)
        best_model = clf.best_estimator_
        print("Best Params:", clf.best_params_)
    else:
        model.fit(X_train_2, y_train_2)
        best_model = model

    y_pred = best_model.predict(X_test_2)

    weighted_f1 = f1_score(y_test_2, y_pred, average='weighted')
    accuracy = accuracy_score(y_test_2, y_pred)

    print(f"Accuracy: {accuracy:.2%}")
    print(f"Weighted F1 Score: {weighted_f1:.4f}")
    print(classification_report(y_test_2, y_pred, zero_division=0))



************** Random Forest *******************
Best Params: {'max_depth': 10, 'n_estimators': 200}
Accuracy: 64.57%
Weighted F1 Score: 0.6349
              precision    recall  f1-score   support

       angry       0.73      0.76      0.75       105
       happy       0.68      0.80      0.73        60
     neutral       0.46      0.47      0.47        57
         sad       0.60      0.28      0.38        32

    accuracy                           0.65       254
   macro avg       0.62      0.58      0.58       254
weighted avg       0.64      0.65      0.63       254


************** Logistic Regression *******************
Best Params: {'C': 10, 'solver': 'lbfgs'}
Accuracy: 64.57%
Weighted F1 Score: 0.6247
              precision    recall  f1-score   support

       angry       0.71      0.79      0.75       105
       happy       0.67      0.87      0.75        60
     neutral       0.45      0.33      0.38        57
         sad       0.59      0.31      0.41        32

    acc

 Ensemble: Naive Bayes + Random Forest

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score

# Initialize models
nb = GaussianNB()
rf = RandomForestClassifier(n_estimators=100, random_state=42)

# Voting classifier (soft = average class probabilities)
ensemble = VotingClassifier(
    estimators=[('nb', nb), ('rf', rf)],
    voting='soft'
)

# Train ensemble
ensemble.fit(X_train_2, y_train_2)

# Predict
y_pred = ensemble.predict(X_test_2)

# Evaluate
print("\n************** Ensemble: Naive Bayes + Random Forest *******************")
print(f"Accuracy: {accuracy_score(y_test_2, y_pred) * 100:.2f}%")
print(f"Weighted F1 Score: {f1_score(y_test_2, y_pred, average='weighted'):.4f}")
print(classification_report(y_test_2, y_pred, zero_division=0))



************** Ensemble: Naive Bayes + Random Forest *******************
Accuracy: 66.14%
Weighted F1 Score: 0.6530
              precision    recall  f1-score   support

       angry       0.78      0.76      0.77       105
       happy       0.68      0.83      0.75        60
     neutral       0.48      0.49      0.49        57
         sad       0.53      0.31      0.39        32

    accuracy                           0.66       254
   macro avg       0.62      0.60      0.60       254
weighted avg       0.66      0.66      0.65       254

